# Web Scraping Homework - Mission to Mars

### NASA Mars News

In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import pandas as pd
import time
import requests
from sqlalchemy import create_engine
from urllib.parse import urlsplit
from selenium import webdriver

In [2]:
# Use splinter to control browser
browser = Browser("chrome", headless=False)

In [3]:
# Visit the web page - moved to a separate cell because the 
# code in the next cell errored when this code was in the same cell
mars_url = 'https://mars.nasa.gov/news/'
browser.visit(mars_url)

In [4]:
# #Visit the web page
# url = 'https://mars.nasa.gov/news/'
# browser.visit(url)

#using bs to write it into html
html = browser.html
soup = bs(html,'html.parser')

news_title = soup.find('div',class_= 'content_title').text
news_paragraph = soup.find('div', class_= 'article_teaser_body').text

print(f"Title: {news_title}")
print(f"Paragraph: {news_paragraph}")


Title: NASA Updates Mars 2020 Mission Environmental Review
Paragraph: NASA and the Department of Energy have completed a more detailed risk analysis for the Mars 2020 rover launch from Florida.


### JPL Mars Space Images - Featured Image

In [5]:
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

In [6]:
#Getting the base url using 
base_url = 'https://www.jpl.nasa.gov'
print(base_url)

https://www.jpl.nasa.gov


In [7]:
#get image url using BeautifulSoup
image_url = browser.html
soup = bs(image_url, 'html.parser')
img_url = soup.find(id='full_image').get('data-fancybox-href')
fullimgurl = base_url + img_url
print(fullimgurl)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17793_ip.jpg


### Mars Weather 

In [8]:
#get mars weather's latest tweet from the website
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [9]:
weather_html = browser.html
soup = bs(weather_html, 'html.parser')
marsweather = soup.find('p', class_= 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
print(marsweather)


InSight sol 353 (2019-11-24) low -100.6ºC (-149.1ºF) high -23.3ºC (-10.0ºF)
winds from the SSE at 5.8 m/s (13.1 mph) gusting to 21.4 m/s (47.9 mph)
pressure at 6.70 hPapic.twitter.com/NPmuRAuvQ7


### Mars Facts

In [10]:
marsfacts_url = 'https://space-facts.com/mars/'

In [11]:
table = pd.read_html(marsfacts_url)
table[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
marsfacts_df = table[1]


In [13]:
marsfacts_html = marsfacts_df.to_html(index=False)
marsfacts_html = marsfacts_html.replace("\n", "")
marsfacts_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Mars - Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <td>Moons:</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <td>Length of Year:</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <td>Temperature:</td>      <td>-153 to 20 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

### Mars Hemispheres

In [14]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

In [15]:
#Getting the base url
hemisphere_base_url = 'https://astrogeology.usgs.gov'
print(hemisphere_base_url)

https://astrogeology.usgs.gov


In [16]:
# scrape images of Mars' hemispheres from the USGS site
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemisphere_results = []

for i in range(1,9,2):
    hemisphere_dict = {}
    
    browser.visit(mars_hemisphere_url)
    time.sleep(1)
    hemispheres_html = browser.html
    hemispheres_soup = bs(hemispheres_html, 'html.parser')
    hemispherenamelinks = hemispheres_soup.find_all('a', class_='product-item')
    hemispherename = hemispherenamelinks[i].text.strip('Enhanced')
    
    linkdetail = browser.find_by_css('a.product-item')
    linkdetail[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    hemisphereimgage_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    hemisphereimgage_soup = bs(hemisphereimgage_html, 'html.parser')
    hemisphereimage_path = hemisphereimgage_soup.find('img')['src']

    print(hemispherename)
    hemisphere_dict['title'] = hemispherename.strip()
    
    print(hemisphereimage_path)
    hemisphere_dict['img_url'] = hemisphereimage_path

    hemisphere_results.append(hemisphere_dict)

Cerberus Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


### Dictionary

In [17]:
# create a dictionary containing the collected data for later use in flask app
mars_info={"news_title":news_title,
           "news_paragraph":news_paragraph,
           "featuredimage_url":fullimgurl,
           "tweet_url":base_url,
           "marsweather":marsweather,
           "marsfacts":marsfacts_url,
           "hemisphereimage_urls":hemisphere_results  
          }
print(mars_info)

{'news_title': 'NASA Updates Mars 2020 Mission Environmental Review', 'news_paragraph': 'NASA and the Department of Energy have completed a more detailed risk analysis for the Mars 2020 rover launch from Florida.', 'featuredimage_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17793_ip.jpg', 'tweet_url': 'https://www.jpl.nasa.gov', 'marsweather': 'InSight sol 353 (2019-11-24) low -100.6ºC (-149.1ºF) high -23.3ºC (-10.0ºF)\nwinds from the SSE at 5.8 m/s (13.1 mph) gusting to 21.4 m/s (47.9 mph)\npressure at 6.70 hPapic.twitter.com/NPmuRAuvQ7', 'marsfacts': 'https://space-facts.com/mars/', 'hemisphereimage_urls': [{'title': 'Cerberus Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astropedia.astro